This notebook look at whether ensemble of traditional NLP and neural NLP will produce good result. This could be very much an overkill, but it would be interesting to explore nontheless. </n>

In summary, traditional NLP with hand crafted features and random forest produced a f1 score of 0.88, whereas the fine-tuned pre-trained BERT model produced an accuracy score of around 94%. The intuition of combining these two models is that hand crafted features very much captures frequencies of certain words or types of words. Pre-train BERT model has a better understanding of the language as a whole, and this could help identify different writing styles.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pickle
from prep_bert import *
from ensemble import *
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig

In [2]:
with open('tokenised-data.pickle', 'rb') as handle:
    df = pickle.load(handle)

In [3]:
df.head()

,url,class_label,class_id,text,ents_rep,vocab,ppo_rep,no_ents_text,verb_present,verb_past,...,ad,prob,lem_text,len,org_count,place_count,time_count,person_count,num_count,ne_count
0,https://www.nytimes.com/2020/08/17/world/afric...,Incident Report,0,advertisement supported by the extremist group...,1.52,0.570312,1.71875,advertisement support extremist group ORG esca...,0.015625,0.0703125,...,0.119792,0,"[advertisement, support, extremist, group, ORG...",384,0.059896,0.057292,0.039062,0.026042,0.015625,0.143229
1,https://www.bbc.com/news/world-europe-54500555,Incident Report,0,belarusian riot police have used water cannon ...,1.41333,0.57716,1.54902,NORP riot police water cannon stun grenade bre...,0.037037,0.0432099,...,0.125,0.00154321,"[NORP, riot, police, water, cannon, stun, gren...",648,0.030864,0.026235,0.024691,0.064815,0.013889,0.121914
2,https://www.aljazeera.com/news/2020/10/12/nago...,Incident Report,0,russian foreign minister calls on armenia and ...,2.2518,0.372881,2.62887,NORP foreign minister call GPE GPE adhere agre...,0.0209847,0.0524617,...,0.0992736,0.00161421,"[NORP, foreign, minister, call, GPE, GPE, adhe...",1239,0.066182,0.092010,0.033091,0.047619,0.012914,0.205811
3,https://counteriedreport.com/roadside-bomb-bla...,Incident Report,0,alshabaab ended 2019 with a truckborne improvi...,1.25,0.75,1.5,GPE end DATE truckborne improvise explosive de...,0.0277778,0.037037,...,0.212963,0,"[GPE, end, DATE, truckborne, improvise, explos...",108,0.027778,0.027778,0.018519,0.000000,0.018519,0.055556
4,https://counteriedreport.com/al-shabaabs-impro...,Incident Report,0,alshabaab ended 2019 with a truckborne improvi...,1.25,0.75,1.5,GPE end DATE truckborne improvise explosive de...,0.0277778,0.037037,...,0.212963,0,"[GPE, end, DATE, truckborne, improvise, explos...",108,0.027778,0.027778,0.018519,0.000000,0.018519,0.055556


## I. Individual model performances

In [3]:
X = df[['text','len', 'org_count', 'place_count', 'time_count', 'person_count', 'num_count', 'ents_rep', 'vocab', 'ppo_rep', 'verb_present', 'verb_past', 'verb', 'modal', 'ad', 'prob']]
y = df[['class_label', 'class_id']]
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, stratify=y, random_state=2020)

In [4]:
X_rf = X_test[['len', 'org_count', 'place_count', 'time_count', 'person_count', 'num_count', 'ents_rep', 'vocab', 'ppo_rep', 'verb_present', 'verb_past', 'verb', 'modal', 'ad', 'prob']]
y_rf = y_test['class_label']


texts = X_test['text'].to_list()
labels = y_test['class_id'].apply(lambda x:int(x))

dataset = BertEncoder(
    tokenizer=BertTokenizer.from_pretrained(
        'bert-base-cased',
        do_lower_case=False),
    input_data=texts
)

tokenized_data = dataset.tokenize(max_len=510)

input_ids, attention_masks = tokenized_data
labels = torch.Tensor(labels.to_list()).long()

dataloader = build_test_dataloaders(
    input_ids=input_ids,
    attention_masks=attention_masks,
    labels=labels
)

In [5]:
rf = pickle.load(open('./model/rf_model.sav', 'rb'))

In [11]:
# Please note the scores here are not the same as those in the traditional NLP notebook.
# The reason is I accidentally ran tranditional NLP notebook random search cv again, 
# so the model there is not the same as the one save here. 
y_pred = rf.predict(X_rf)
print(classification_report(y_rf, y_pred))
print(confusion_matrix(y_rf, y_pred))

                   precision    recall  f1-score   support

Analytical Report       0.88      0.93      0.90        15
  Incident Report       0.94      0.83      0.88        18
   Profile Report       0.78      0.90      0.84        20
 Situation Report       0.94      0.85      0.89        20

         accuracy                           0.88        73
        macro avg       0.88      0.88      0.88        73
     weighted avg       0.88      0.88      0.88        73

[[14  0  0  1]
 [ 0 15  3  0]
 [ 1  1 18  0]
 [ 1  0  2 17]]


In [6]:
bert_state_dict = torch.load('./model/trained_model.pt')
bert = BertForSequenceClassification.from_pretrained(
    'bert-base-cased',
    state_dict=bert_state_dict,
    num_labels = 4,
    output_attentions=False,
    output_hidden_states=False
)

In [7]:
y_b_pred, _ = bert_predict(bert, dataloader)

[[ 6.0034976  -1.4367361  -1.1977938  -2.1563373 ]
 [ 6.12242    -1.1885549  -1.7572205  -1.7771249 ]
 [ 1.9933681  -1.9661794   3.7339196  -3.634086  ]
 [-2.5564435  -1.8960001   5.8017936  -2.1922736 ]
 [-2.3510206  -1.8906516   5.90642    -2.382116  ]
 [ 6.0853243  -1.0437504  -1.8861271  -1.67723   ]
 [-1.4554386   4.9191628  -1.9625443  -1.5986322 ]
 [ 5.9895005  -1.2478404  -1.83326    -1.6556695 ]
 [-2.6087663  -1.9404153   5.72297    -1.9552666 ]
 [-2.405675   -1.869924    5.8454823  -2.2820027 ]
 [-1.0841643  -1.3103201   4.953755   -2.838082  ]
 [-2.2074788  -0.26162842 -2.205988    5.0521746 ]
 [-1.8595176  -1.3526336  -1.7825524   5.192944  ]
 [-1.9011126  -1.2507565  -1.8456715   5.078395  ]
 [ 6.081219   -1.301969   -1.7657568  -1.609917  ]
 [-1.5199683   4.8878284  -1.9637839  -1.4974451 ]
 [-1.8598015  -0.94654393 -2.0823646   5.2278214 ]
 [-2.4640503  -1.8799112   5.871163   -2.378856  ]
 [-1.945038   -0.8934976  -2.1046457   5.2174597 ]
 [ 6.098221   -1.3404071  -1.65

In [8]:
cls_names = ['Incident Report', 'Situation Report', 'Profile report', 'Analytical report']
print(classification_report(labels, y_b_pred, target_names=cls_names))
print(confusion_matrix(labels, y_b_pred))

                   precision    recall  f1-score   support

  Incident Report       0.81      0.94      0.87        18
 Situation Report       0.95      0.95      0.95        20
   Profile report       1.00      0.80      0.89        20
Analytical report       0.88      0.93      0.90        15

         accuracy                           0.90        73
        macro avg       0.91      0.91      0.90        73
     weighted avg       0.91      0.90      0.90        73

[[17  0  0  1]
 [ 0 19  0  1]
 [ 4  0 16  0]
 [ 0  1  0 14]]


## II. Ensemble
Considering that random forest took little time to train, the result is really good. However, the purpose of this notebook is to look at if ensemble both models will produce better result. </n>

First, let's look at if both models predict the same labels.

In [9]:
y_b_pred = [int(x) for x in y_b_pred]

In [12]:
print('Random forest misclassified groud truth:', y_rf[y_rf!=y_pred])
print('Random forest misclassified predicted:', y_pred[y_rf!=y_pred])
print()
y_nn = labels.numpy()
print('BERT model misclassified ground truth', y_test['class_label'][y_nn!=y_b_pred])
print('BERT model misclassified predicted', np.array(y_b_pred)[y_nn!=y_b_pred])

Random forest misclassified groud truth: 96        Profile Report
81      Situation Report
86       Incident Report
48     Analytical Report
148       Profile Report
83       Incident Report
323     Situation Report
281     Situation Report
259      Incident Report
Name: class_label, dtype: object
Random forest misclassified predicted: ['Analytical Report' 'Analytical Report' 'Profile Report'
 'Situation Report' 'Incident Report' 'Profile Report' 'Profile Report'
 'Profile Report' 'Profile Report']

BERT model misclassified ground truth 140       Profile Report
159       Profile Report
48     Analytical Report
148       Profile Report
281     Situation Report
149       Profile Report
259      Incident Report
Name: class_label, dtype: object
BERT model misclassified predicted [0 0 1 0 3 0 3]


There are some overlap (48, 148, 281, 259) but more misclassified cases of different samples. Even the overlapped misclassification, the predicitons are different between these two models. There is now a good intuition for ensemble.

In [ ]:
ensemble = NLPEnsemble(
    traditional_nlp=rf,
    nn_nlp=bert
)

pred = ensemble.predict(X_rf, dataloader)